![Logo AML](https://github.com/nicolastibata/MINE_4206_AML_202610/blob/main/docs/logo.jpg?raw=true)

# **Laboratorio 1 - Sesión 2: Regresión Polinomial y Regularizada**
**Tutor: Nicolás Tibatá**

## **Tabla de Contenido**

[Contexto y objetivos](#scrollTo=kgcjzQ76ODRS)<br>
[1. Introducción de los datos](#scrollTo=2dNwUTBZO8OE)<br>
[2. Preparación y Modelamiento](#scrollTo=SlV42CI6PBoQ)<br>
[3. Preguntas](#scrollTo=OCwUBTvxPRdS)<br>

### **Contexto y Objetivos**

Reducir las emisiones de CO2 es crucial para mitigar el cambio climático y minimizar sus efectos nocivos sobre el medio ambiente y el bienestar humano. Esto implica hacer la transición a fuentes de energía más limpias y renovables, mejorar la eficiencia energética, adoptar prácticas sostenibles y promover esfuerzos de conservación. El set de datos reúne información de emisiones de automotores e información de consumo de combustible.

- Ver las implicaciones de las regularizaciones y relaciones polinomiales.
- Aplicar pipelines para el preprocesamiento de datos, aplicar diferentes tipos de escaladores.
- Realizar una busqueda del mejor modelo.

**Datos:** [CO2 Emissions](https://www.kaggle.com/datasets/bhuviranga/co2-emissions)

**Diccionario**
| Columna | Descripción |
| :--- | :--- |
| **Make** | Nombre del fabricante de automotores. |
| **Model** | Modelo del automotor. |
| **Vehicle Class** | Clase de vehículo. |
| **Engine Size (L)** | Tamaño del motor. Las unidades están expresadas en litros. |
| **Transmission** | Tipo de transmisión del vehículo, automática o manual. |
| **Fuel Type** | Tipo de combustible: *Regular Gasoline* (X), *Premium Gasoline* (Z), *Ethanol* (E), *Diesel* (D), *Natural Gas* (N). |
| **Fuel Consumption City (L/100km)** | Consumo del vehículo en ciudad. Las unidades están expresadas en litros por kilómetro. |
| **Fuel Consumption Hwy (L/100 km)** | Consumo del vehículo en carretera. Las unidades están expresadas en litros por kilómetro. |
| **Fuel Consumption Comb (L/100 km)** | Consumo del vehículo en ciudad y en carretera. Las unidades están expresadas en litros por kilómetro.|
| **Fuel Consumption Comb (mpg)** | Consumo del vehículo en ciudad y en carretera. Las unidades están expresadas en millas por galón. |
| **CO2 Emissions(g/km)** | Cantidad de gramos emitidos de C02 por kilometro. |

### **1. Introducción a los datos**

In [ ]:
# Importamos o instalamos librerias necesarias
!pip install ydata-profiling -q
from ydata_profiling import ProfileReport
# Acceso a credenciales
import os
from google.colab import files
from google.colab import userdata
# Manejo de datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Modelamiento
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer, make_column_selector

Cargamos los datos directamente de kaggle con las credenciales personales, pueden consultar cómo funciona [acá](https://www.kaggle.com/discussions/general/74235#2580958).

In [ ]:
from google.colab import userdata
userdata.get('adrianamariarios')

In [ ]:
os.environ["KAGGLE_KEY"] = userdata.get('adrianamariarios')
os.environ["KAGGLE_USERNAME"] = userdata.get('adrianamariarios')

!kaggle datasets download -d bhuviranga/co2-emissions
!unzip "co2-emissions.zip"

In [ ]:
data = pd.read_csv('CO2 Emissions.csv')
data.info()
data.describe()

In [ ]:
data

### **2. Preparación y Modelamiento**

Antes de preparar y modelar los datos, veamos un poco cómo se constituyen

In [ ]:
ProfileReport(data)

**¿Cuáles son los insights que encontramos?**
- `Model` es una variable con muchos valores únicos (2053) lo cual hace que dicha variable se considere con una alta cardinalidad, esto genera ruido estadístico a nuestros modelos o problemas de alta dimensionalidad.
- `Vehicle Type` es una variable que por sus características de cilindraje, engine size o el consumo son intrisecamente relacionadas, por lo tanto podemos descartarla.
- Encontramos registros duplicados por lo tanto debemos eliminarlos.
- Encontramos datos ausentes dentro de las columnas, en este caso en nuestro pipeline aplicamos un imputador de ausencias.

#### **Preparación**

In [ ]:
# 1. Separar los datos en entrenamiento y test -> SIEMPRE
train_data, test_data = train_test_split(data, test_size=0.2, random_state=71)

In [ ]:
# Variables a eliminar
drop_fields = ['Model', 'Vehicle Class']
# Variable objetivo
target_feature = 'CO2 Emissions(g/km)'
# Variables categoricas
categorical_features = ['Make', 'Transmission', 'Fuel Type']

# Preprocesamiento
def preprocess(df):
  print(f"Total de filas duplicadas eliminadas: {df.duplicated().sum().sum()}")
  df = df.drop_duplicates() # Eliminamos duplicados
  df = df.drop(drop_fields, axis = 1) # Eliminamos columnas que no vamos a usar
  X_data, y_variable = df.drop([target_feature], axis=1), df[target_feature] # Separamos nuestro target de las variables explicativas
  return X_data, y_variable

In [ ]:
X_train, y_train = preprocess(train_data)
display(X_train)
display(y_train)

In [ ]:
# Definamos las variables numericas
num_features = list(set(X_train.columns) - set(categorical_features))
num_features

##### **Pipeline**

¿Qué es un pipeline? El pipeline permite ensamblar varios pasos y validarlos de forma cruzada mientras se ajustan diferentes parámetros. Esto nos sirve para replicar nuestro procesamiento a diferentes set de datos o diferentes algoritmos sin tener que pasar otra vez por todas las lineas de código.

In [ ]:
# 1. Funciones complementarias

# Funcion que hace rename a las categorias
def fix_cat_values(df):
  for column in df.columns:
    if column == 'Fuel Type':
      fuel_labels = {'X': 'Regular Gasoline', 'Z': 'Premium Gasoline', 'E': 'Ethanol', 'D': 'Diesel', 'N': 'Natural Gas'}
      df['Fuel Type'] = df['Fuel Type'].replace(fuel_labels)
    elif column == 'Transmission':
      df['Transmission'] = df['Transmission'].apply(lambda x: 'Automatic' if x.startswith('A') else 'Manual')
  return df

In [ ]:
# 2. Construcción del Pipeline

# num_transformer reemplaza los valores ausentes por el mean para las variables numéricas
num_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='mean'))
    ]
)

# cat_transformer aplica de primer paso nuestra funcion complementaria y luego sobre ese resultado aplica OneHotEncoder
cat_transformer = Pipeline(
    steps=[
        ('imputer', FunctionTransformer(fix_cat_values, validate=False)),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]
)

Una vez que hemos definido los transformadores, definimos los pasos (`steps`), que permitirán construir y ejecutar el Pipeline. Los pasos son los siguientes:

1.   `num`, incluye el transformador numérico y la relación de las columnas numéricas haciendo uso de `make_column_selector`.
2.   `cat`, incluye el transformador categórico y la relación de dichas columnas haciendo uso de `make_column_selector`.

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, make_column_selector(dtype_include=np.number)),
        ("cat", cat_transformer, make_column_selector(dtype_include=object))
    ]
)

In [ ]:
# Visualizamos nuestro pipeline
pipe = Pipeline(steps=[
    ('column_transformer', preprocessor)
])

pipe

In [ ]:
pipe.fit_transform(X_train)

In [ ]:
cat_column_names = pipe['column_transformer'].transformers_[1][1][1].get_feature_names_out()
num_column_names = pipe['column_transformer'].transformers_[0][1].feature_names_in_
col_names = list(num_column_names) + list(cat_column_names)

col_names

In [ ]:
pd.DataFrame(pipe.fit_transform(X_train).toarray(), columns=col_names)

Si quieres saber más sobre como funciona el OneHotEncoder: o sus alternativas, lo puedes consultar [acá](https://scikit--learn-org.translate.goog/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html?_x_tr_sl=en&_x_tr_tl=es&_x_tr_hl=es&_x_tr_pto=tc)

#### **Modelamiento**

##### **Modelo: Regresión Lineal**

In [ ]:
estimators_lr = [
        ('transform',preprocessor),
        ('regression', LinearRegression())
]

pipe_lr = Pipeline(estimators_lr)

pipe_lr.fit(X_train, y_train)

In [ ]:
coef_lr = dict(zip(col_names, pipe_lr["regression"].coef_))
for k,v in coef_lr.items():
    print(f'{k} = {v:,.2f}')

In [ ]:
# Evaluemos el modelo
# 1. Apliquemos los mismos pasos al conjunto de test
X_test, y_test = preprocess(test_data)
display(X_test)
display(y_test)

In [ ]:
y_pred_test_lr = pipe_lr.predict(X_test)
y_pred_test_lr

Así de facil con `pipe_lr` aplicamos los pasos del preprocesamiento y además evaluamos nuestro modelo con el conjunto de test

In [ ]:
n,p = X_test.shape

print('------------ Regresión Lineal ------------')
print("Sum of squares (MSE): %.2f" % mean_squared_error(y_test, y_pred_test_lr))
print("Root of sum of squares (RMSE): %.2f" % mean_squared_error(y_test, y_pred_test_lr) ** (1/2))
print("R2-score: %.5f" % r2_score(y_test, y_pred_test_lr) )
print("Adj R2-score: %.5f" % ( 1-(1-r2_score(y_test, y_pred_test_lr))*(n-1)/(n-p-1)) )

In [ ]:
fig, axs = plt.subplots(1,figsize=(14,7))

xvals = list(range(len(y_test[:50])))
axs.plot(xvals, y_pred_test_lr[:50],'bo-', label='Predicción')
axs.plot(xvals, y_test[:50],'ro-', label='Real')

axs.set(title='Modelo con Regresión Lineal', ylabel=y_test.name)
axs.legend()

plt.tight_layout()
plt.show()

##### **Modelo: Regresión Polinomial**

In [ ]:
estimators_poly = [
        ('transform',preprocessor),
        ('poly',PolynomialFeatures(degree=3)),
        ('regression', LinearRegression())
]

pipe_poly = Pipeline(estimators_poly)

pipe_poly.fit(X_train, y_train)

In [ ]:
coef_lr = dict(zip(col_names, pipe_poly["regression"].coef_))
for k,v in coef_lr.items():
    print(f'{k} = {v:,.2f}')

In [ ]:
# Evaluamos nuestro modelo con test
y_pred_test_poly = pipe_poly.predict(X_test)
y_pred_test_poly

In [ ]:
n,p = X_test.shape

print('------------ Regresión con Transformación Polinomial ------------')
print("Sum of squares (MSE): %.2f" % mean_squared_error(y_test, y_pred_test_poly))
print("Root of sum of squares (RMSE): %.2f" % mean_squared_error(y_test, y_pred_test_poly) ** (1/2))
print("R2-score: %.5f" % r2_score(y_test, y_pred_test_poly) )
print("Adj R2-score: %.5f" % ( 1-(1-r2_score(y_test, y_pred_test_poly))*(n-1)/(n-p-1)) )

In [ ]:
fig, axs = plt.subplots(1,figsize=(14,7))

xvals = list(range(len(y_test[:50])))
axs.plot(xvals, y_pred_test_poly[:50],'bo-', label='Predicción')
axs.plot(xvals, y_test[:50],'ro-', label='Real')

axs.set(title='Modelo con Transformación Polinomial', ylabel=y_test.name)
axs.legend()

plt.tight_layout()
plt.show()

##### **Modelo: Regresión Lineal con regularización L1**

In [ ]:
estimators_lasso = [
        ('transform',preprocessor),
        ('lasso', Lasso(alpha=8))
]

pipe_lasso = Pipeline(estimators_lasso)

pipe_lasso.fit(X_train, y_train)

In [ ]:
coef_lr = dict(zip(col_names, pipe_lasso["lasso"].coef_))
for k,v in coef_lr.items():
    print(f'{k} = {v:,.2f}')

In [ ]:
# Evaluamos nuestro modelo
y_pred_test_lasso = pipe_lasso.predict(X_test)
y_pred_test_lasso

In [ ]:
n,p = X_test.shape

print('------------ Regresión Lasso ------------')
print("Sum of squares (MSE): %.2f" % mean_squared_error(y_test, y_pred_test_lasso))
print("Root of sum of squares (RMSE): %.2f" % mean_squared_error(y_test, y_pred_test_lasso) ** (1/2))
print("R2-score: %.5f" % r2_score(y_test, y_pred_test_lasso) )
print("Adj R2-score: %.5f" % ( 1-(1-r2_score(y_test, y_pred_test_lasso))*(n-1)/(n-p-1)) )

In [ ]:
fig, axs = plt.subplots(1,figsize=(14,7))

xvals = list(range(len(y_test[:50])))
axs.plot(xvals, y_pred_test_lasso[:50],'bo-', label='Predicción')
axs.plot(xvals, y_test[:50],'ro-', label='Real')

axs.set(title='Modelo con Regularización Lasso', ylabel=y_test.name)
axs.legend()

plt.tight_layout()
plt.show()

##### **Búsqueda del mejor modelo**

En este caso no solo usaremos un único modelo, sino diferentes combinaciones posibles. Para esto usamos Grid Search CV

In [ ]:
estimators_best = [
        ('transform', preprocessor),
        ('polinomial',PolynomialFeatures()),
        ('regression', Lasso())
]

pipe_best = Pipeline(estimators_best)

# Parametros de la busqueda
parameters = {
              'polinomial__degree':[2,3],
              'regression__alpha': [0.01, 1],
              'transform__num': [StandardScaler(), MinMaxScaler(), 'passthrough'],
              'transform__cat': [OneHotEncoder(handle_unknown='ignore'), OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=0)]
              }

grid_search = GridSearchCV(pipe_best, parameters, verbose=2, scoring='neg_mean_squared_error', cv=5)

In [ ]:
%%time
grid_search.fit(X_train, y_train)

In [ ]:
best_model = grid_search.best_estimator_

pd.DataFrame(grid_search.cv_results_)

In [ ]:
grid_search.best_params_

In [ ]:
# Evaluamos nuestro modelo
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)
y_pred_train

In [ ]:
n,p = X_train.shape

print('------------ Regresión Lasso con entrenamiento------------')
print("Residual sum of squares (MSE): %.2f" % mean_squared_error(y_train, y_pred_train))
print("Root of sum of squares (RMSE): %.2f" % mean_squared_error(y_train, y_pred_train) ** (1/2))
print("R2-score: %.5f" % r2_score(y_train, y_pred_train) )
print("Adj R2-score: %.5f" % ( 1-(1-r2_score(y_train, y_pred_train))*(n-1)/(n-p-1)) )

n,p = X_test.shape

print('------------ Regresión Lasso con evaluación ------------')
print("Residual sum of squares (MSE): %.2f" % mean_squared_error(y_test, y_pred_test))
print("Root of sum of squares (RMSE): %.2f" % mean_squared_error(y_test, y_pred_test) ** (1/2))
print("R2-score: %.5f" % r2_score(y_test, y_pred_test) )
print("Adj R2-score: %.5f" % ( 1-(1-r2_score(y_test, y_pred_test))*(n-1)/(n-p-1)) )

# No hay indicios de sobreajuste

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,figsize=(20,10))
a = 1000
b= 1080

xvals = list(range(b-a))
axs[0].plot(xvals, y_pred_train[a:b],'bo-', label='Predicción')
axs[0].plot(xvals, y_train[a:b],'ro-', label='Real')

axs[1].plot(xvals, y_pred_test[a:b],'bo-', label='Predicción')
axs[1].plot(xvals, y_test[a:b],'ro-', label='Real')

axs[0].set(title='Predicción con Regresión Lasso con CV - Entrenamiento', ylabel=y_train.name)
axs[0].legend()

axs[1].set(title='Predicción con Regresión Lasso con CV - Evaluación', ylabel=y_train.name)
axs[1].legend()

plt.tight_layout()
plt.show()

Ya con nuestro mejor modelo encontrado y teniendo un rendimiento esperado en producción, podremos determinar la importancia de las variables del modelo. Para ello se obtuvo el siguiente resultado.

In [ ]:
lasso_model = best_model['regression']
trans_df = best_model['transform'].transform(X_test)
fake_df = best_model['polinomial'].transform(trans_df)

cat_names = best_model['transform'].transformers_[1][1].get_feature_names_out()
num_names = best_model['transform'].transformers_[0][2]#.feature_names_in_
col_names = list(num_names) + list(cat_names)

print(f'Intercepto: {lasso_model.intercept_}')
coef = list(zip(['Intercepto'] + list(col_names), [lasso_model.intercept_] + list(lasso_model.coef_)))
coef = pd.DataFrame(coef,columns=['Variable','Parámetro'])
coef

### **3. Preguntas**

1. Cree un pipeline con el algoritmo de su seleccion donde incluya un minmaxScaler a las variables que aplique

```
num_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', MinMaxScaler())  # Adicion acá
    ]
)

cat_transformer = Pipeline(
    steps=[
        ('imputer', FunctionTransformer(fix_cat_values, validate=False)),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, make_column_selector(dtype_include=np.number)),
        ("cat", cat_transformer, make_column_selector(dtype_include=object))
    ]
)
```

2. ¿Cuáles son las diferencias entre regresión lineal con transformación polinomial y regresión regularizada Lasso?
3. ¿Qué otro espacio de búsqueda o hiperparámetros serían buenos de buscar en este modelo?.
4. La columna `Make` puede considerarse una variable de alta cardinalidad. ¿Cómo serían los resultados si se construye un modelo sin esta variable?